In [12]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter,Constant)
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
#from dike_model_function import DikeNetwork  # @UnresolvedImport

def sum_over(*args):
    return sum(args)

In [14]:
from ema_workbench import (Model, RealParameter, ScalarOutcome, SequentialEvaluator,
                           MultiprocessingEvaluator, ema_logging,
                           Constant, Policy, Scenario)

ema_logging.log_to_stderr(ema_logging.INFO)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench import ScalarOutcome
from ema_workbench.util import ema_logging
import time
from problem_formulation_V2_0 import get_model_for_actor_problem_formulation

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [15]:
#generate the model --Change the argument for different actor formulations
model, planning_steps = get_model_for_actor_problem_formulation(1, outcome_type='scalar')

In [16]:
#for unc in dike_model.uncertainties:
 #   print(repr(unc))

In [17]:
#for unc in dike_model.uncertainties:
#    print(repr(unc))
    
uncertainties = model.uncertainties

import copy
uncertainties = copy.deepcopy(model.uncertainties)

In [18]:
#for policy in model.levers: 
#    print(repr(policy))
levers = model.levers 

import copy
levers = copy.deepcopy(model.levers)

In [19]:
for outcome in model.outcomes:
   print(repr(outcome))

ScalarOutcome('Expected Annual Damage Gelderland', variable_name=['A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.3_Expected Annual Damage 0', 'A.3_Expected Annual Damage 1', 'A.3_Expected Annual Damage 2'], function=<function sum_over at 0x0000022C939C7A60>)
ScalarOutcome('Dike Investment Costs Gelderland', variable_name=['A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2', 'A.3_Dike Investment Costs 0', 'A.3_Dike Investment Costs 1', 'A.3_Dike Investment Costs 2'], function=<function sum_over at 0x0000022C939C7A60>)
ScalarOutcome('Expected Number of Deaths in Gelderland', variable_name=['A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2', 'A.2_Expected Numbe

In [20]:
#create reference case of policy levers all set to 0
levs = []
for lev in model.levers:
    levs.append(lev)\

do_nothing_policy = Policy("do nothing", **{lever.name:0 for lever in levs})

# Build a user-defined scenario and policy:
reference_values = {'Bmax': 175, 'Brate': 1.5, 'pfail': 0.5,
                        'ID flood wave shape': 4, 'planning steps': 2}
reference_values.update({'discount rate {}'.format(n): 3.5 for n in planning_steps})
scen1 = {}

for key in model.uncertainties:
    name_split = key.name.split('_')

    if len(name_split) == 1:
        scen1.update({key.name: reference_values[key.name]})
    else:
        scen1.update({key.name: reference_values[name_split[1]]})

ref_scenario = Scenario('reference', **scen1)

#no dike increase, no warning, none of the rfr
zero_policy = {'DaysToThreat': 0}
zero_policy.update({'DikeIncrease {}'.format(n): 0 for n in planning_steps})
zero_policy.update({'RfR {}'.format(n): 0 for n in planning_steps})
pol0 = {}

for key in model.levers:
    s1, s2 = key.name.split('_')
    pol0.update({key.name: zero_policy[s2]})

policy0 = Policy('Policy 0', **pol0)
    
#define convergence metrics --COMPUTATIONALLY EXPENSIVE SO COMMENT OUT WHERE NECESSARY
#specifying min and max values...play around with this max!!
convergence_metrics = [HyperVolume(minimum=[0,0,0,0,0,0,0,0], maximum=[2e09,2e08,10,2e09,2e08,10,2e09,10000]), EpsilonProgress()]

In [21]:
#with both searchover = 'uncertainties' then = 'levers'...but then change the refernece!!
with MultiprocessingEvaluator(model) as evaluator:
    results, convergence = evaluator.optimize(nfe=1000, searchover='levers', logging_freq = 100,
                                              epsilons=[100000,8000,0.0001,100000,8000,0.0001,80000,8],convergence = convergence_metrics, reference=ref_scenario)

[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/1000 nfe
[MainProcess/INFO] optimization completed, found 159 solutions
[MainProcess/INFO] terminating pool


In [11]:
outcomes = results.iloc[:,31:39]

In [20]:
outcomes

,Expected Annual Damage Gelderland,Dike Investment Costs Gelderland,Expected Number of Deaths in Gelderland,Expected Annual Damage Overijssel,Dike Investment Costs Overijssel,Expected Number of Deaths in Overijssel,RfR Total Costs,Expected Total Evacuation Costs
0,4.112340e+06,4.669789e+08,0.000580,3.309499e+06,1.179711e+08,0.000450,6.721000e+08,610.385654
1,5.411423e+06,3.437332e+08,0.006364,0.000000e+00,1.429343e+08,0.000000,1.000800e+09,0.000000
2,0.000000e+00,4.086696e+08,0.000000,1.954952e+07,8.965036e+07,0.003285,8.975000e+08,1208.271731
3,0.000000e+00,3.498244e+08,0.000000,0.000000e+00,1.993114e+08,0.000000,1.000800e+09,0.000000
4,2.465575e+06,4.416655e+08,0.000814,2.154725e+07,8.626535e+07,0.003669,7.045000e+08,1424.010196
5,0.000000e+00,4.394012e+08,0.000000,1.624002e+07,1.595995e+08,0.006535,6.721000e+08,605.799322
6,2.469740e+06,6.133662e+08,0.000817,1.250587e+06,1.181370e+08,0.000129,6.662000e+08,186.060645
7,2.587058e+06,4.516886e+08,0.001091,0.000000e+00,1.196753e+08,0.000000,1.066200e+09,82.917137
8,0.000000e+00,4.201774e+08,0.000000,0.000000e+00,1.768764e+08,0.000000,5.448000e+08,0.000000
9,0.000000e+00,4.633072e+08,0.000000,2.232063e+06,1.120340e+08,0.000178,9.223000e+08,181.381936


In [ ]:
#Plot the convergence statistics
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')
ax2.plot(convergence.nfe, convergence.hypervolume)
ax2.set_ylabel('hypervolume')

ax1.set_xlabel('number of function evaluations')
ax2.set_xlabel('number of function evaluations')
plt.show()

In [ ]:
#when we achieve convergence, check the results  only for 3 objectives at a time
#3d scatterplot for pareto optimal solutions
from mpl_toolkits.mplot3d import Axes3D  

ax = fig.add_subplot(111, projection='3d')
ax.scatter??
ax.scatter(#numpy array here)
ax.set_xlabel('Expected Annual Damage')
ax.set_ylabel('Total Investment Costs')
ax.set_zlabel('Expected Number of Deaths')
#plt.show()

In [11]:
outcomes

NameError: name 'outcomes' is not defined

In [ ]:
#evaluate trade offs between the three objectives
from ema_workbench.analysis import parcoords
limits = parcoords.get_limits(outcomes)
axes = parcoords.ParallelAxes(limits)
axes.plot(outcomes)

plt.show()
